In [7]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
# py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

pd.set_option('display.max_columns', None)

In [2]:
from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

서울시 골목상권 프로파일 정보(상권-추정매출).csv
서울시 골목상권 프로파일 정보(상권-집객시설).csv
서울시 골목상권 프로파일 정보(상권-추정유동인구).csv
서울시 골목상권 프로파일링 정보(상권-점포).csv
서울시 골목상권 프로파일링 정보(상권-아파트).csv
서울시 골목상권 프로파일링 정보(상권-지수지표).csv
서울시 골목상권 프로파일링 정보(상권-소득소비).csv
서울시 골목상권 프로파일링 정보(상권-상주인구).csv
서울시 골목상권 프로파일링 정보(상권-직장인구).csv
서울시 골목상권 프로파일링 정보(상권배후지-점포).csv
서울시 골목상권 프로파일링 정보(상권배후지-아파트).csv
서울시 골목상권 프로파일링 정보(상권배후지-소득소비).csv
서울시 골목상권 프로파일링 정보(상권배후지-상주인구).csv
서울시 골목상권 프로파일링 정보(상권배후지-추정매출).csv
서울시 골목상권 프로파일링 정보(상권배후지-직장인구).csv
서울시 골목상권 프로파일링 정보(상권배후지-집객시설).csv
서울시 골목상권 프로파일링 정보(상권배후지-추정유동인구).csv



In [18]:
market_c_expSale = pd.read_csv("./data/서울시 골목상권 프로파일 정보(상권-추정매출).csv", parse_dates=["기준_년월_코드"])
market_c_attractFacility = pd.read_csv("./data/서울시 골목상권 프로파일 정보(상권-집객시설).csv", parse_dates=["기준_년월_코드"])
market_c_expFloatingPopulation = pd.read_csv("./data/서울시 골목상권 프로파일 정보(상권-추정유동인구).csv", parse_dates=["기준_년월_코드"])
market_c_store = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-점포).csv", parse_dates=["기준_년월_코드"])
market_c_apartment = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-아파트).csv", parse_dates=["기준_년월_코드"])
market_c_indiIndex = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-지수지표).csv", parse_dates=["기준_년월_코드"])
market_c_incomConsum = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-소득소비).csv", parse_dates=["기준_년월_코드"])
market_c_residPopulation = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-상주인구).csv", parse_dates=["기준년월 코드"])
market_c_workPopulation = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권-직장인구).csv", parse_dates=["기준_년월_코드"])
market_nc_store = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-점포).csv", parse_dates=["기준 년월 코드"])
market_nc_apartment = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-아파트).csv", parse_dates=["기준년월 코드"])
market_nc_incomConsum = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-소득소비).csv", parse_dates=["기준년월코드"])
market_nc_residPopulation = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-상주인구).csv", parse_dates=["기준년월 코드"])
market_nc_expSale = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-추정매출).csv", parse_dates=["기준_년월_코드"])
market_nc_workPopulation = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-직장인구).csv", parse_dates=["기준_년월_코드"])
market_nc_attractFacility = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-집객시설).csv", parse_dates=["기준_년월_코드"])
market_nc_expFloatingPopulation = pd.read_csv("./data/서울시 골목상권 프로파일링 정보(상권배후지-추정유동인구).csv", parse_dates=["기준_년월_코드"])
print("[1] : "+str(market_c_expSale.shape))
print("[2] : "+str(market_c_attractFacility.shape))
print("[3] : "+str(market_c_expFloatingPopulation.shape))
print("[4] : "+str(market_c_store.shape))
print("[5] : "+str(market_c_apartment.shape))
print("[6] : "+str(market_c_indiIndex.shape))
print("[7] : "+str(market_c_incomConsum.shape))
print("[8] : "+str(market_c_residPopulation.shape))
print("[9] : "+str(market_c_workPopulation.shape))
print("[10] : "+str(market_nc_store.shape))
print("[11] : "+str(market_nc_apartment.shape))
print("[12] : "+str(market_nc_incomConsum.shape))
print("[13] : "+str(market_nc_residPopulation.shape))
print("[14] : "+str(market_nc_expSale.shape))
print("[15] : "+str(market_nc_workPopulation.shape))
print("[16] : "+str(market_nc_attractFacility.shape))
print("[17] : "+str(market_nc_expFloatingPopulation.shape))

[1] : (23425, 83)
[2] : (30832, 23)
[3] : (6048, 529)
[4] : (72877, 11)
[5] : (40942, 18)
[6] : (262020, 17)
[7] : (57459, 15)
[8] : (56542, 27)
[9] : (57144, 24)
[10] : (39346, 11)
[11] : (57204, 18)
[12] : (59245, 15)
[13] : (57230, 27)
[14] : (39030, 83)
[15] : (45569, 24)
[16] : (32174, 23)
[17] : (6048, 528)


In [23]:
market_c_expSale.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,평균_영업_개월_수,1년이하_생존_율,1~2년_생존_율,2~3년_생존_율,3~5년_생존_율,5년이상_생존_율,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,201706,13651,도봉로152가길,CS300007,의류점,6.0,NaN,NaN,NaN,NaN,NaN,10265330,432,66.9,33.1,11.1,10.0,11.1,21.3,13.4,20.1,13.0,0.0,3.0,38.9,41.9,16.2,0.0,27.5,72.5,0.0,4.9,9.3,24.1,22.9,38.9,7403779,2861551,919090,905814,1117442,2443313,2018120,1713973,1147578,0,545834,3738198,4198546,1782752,0,2813002,7452328,0,361245,835045,2710142,2434938,3923960,289,143,48,43,48,92,58,87,56,0,13,168,181,70,0,119,313,0,21,40,104,99,168,1


In [24]:
market_c_attractFacility.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,201707,13182,백제고분로34길,10,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [25]:
market_c_expFloatingPopulation.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_1_유동인구_수,시간대_2_유동인구_수,시간대_3_유동인구_수,시간대_4_유동인구_수,시간대_5_유동인구_수,시간대_6_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수,남성연령대_10_월요일시간대_1_유동인구_수,남성연령대_10_월요일시간대_2_유동인구_수,남성연령대_10_월요일시간대_3_유동인구_수,남성연령대_10_월요일시간대_4_유동인구_수,남성연령대_10_월요일시간대_5_유동인구_수,남성연령대_10_월요일시간대_6_유동인구_수,남성연령대_10_화요일시간대_1_유동인구_수,남성연령대_10_화요일시간대_2_유동인구_수,남성연령대_10_화요일시간대_3_유동인구_수,남성연령대_10_화요일시간대_4_유동인구_수,남성연령대_10_화요일시간대_5_유동인구_수,남성연령대_10_화요일시간대_6_유동인구_수,남성연령대_10_수요일시간대_1_유동인구_수,남성연령대_10_수요일시간대_2_유동인구_수,남성연령대_10_수요일시간대_3_유동인구_수,남성연령대_10_수요일시간대_4_유동인구_수,남성연령대_10_수요일시간대_5_유동인구_수,남성연령대_10_수요일시간대_6_유동인구_수,남성연령대_10_목요일시간대_1_유동인구_수,남성연령대_10_목요일시간대_2_유동인구_수,남성연령대_10_목요일시간대_3_유동인구_수,남성연령대_10_목요일시간대_4_유동인구_수,남성연령대_10_목요일시간대_5_유동인구_수,남성연령대_10_목요일시간대_6_유동인구_수,남성연령대_10_금요일시간대_1_유동인구_수,남성연령대_10_금요일시간대_2_유동인구_수,남성연령대_10_금요일시간대_3_유동인구_수,남성연령대_10_금요일시간대_4_유동인구_수,남성연령대_10_금요일시간대_5_유동인구_수,남성연령대_10_금요일시간대_6_유동인구_수,남성연령대_10_토요일시간대_1_유동인구_수,남성연령대_10_토요일시간대_2_유동인구_수,남성연령대_10_토요일시간대_3_유동인구_수,남성연령대_10_토요일시간대_4_유동인구_수,남성연령대_10_토요일시간대_5_유동인구_수,남성연령대_10_토요일시간대_6_유동인구_수,남성연령대_10_일요일시간대_1_유동인구_수,남성연령대_10_일요일시간대_2_유동인구_수,남성연령대_10_일요일시간대_3_유동인구_수,남성연령대_10_일요일시간대_4_유동인구_수,남성연령대_10_일요일시간대_5_유동인구_수,남성연령대_10_일요일시간대_6_유동인구_수,남성연령대_20_월요일시간대_1_유동인구_수,남성연령대_20_월요일시간대_2_유동인구_수,남성연령대_20_월요일시간대_3_유동인구_수,남성연령대_20_월요일시간대_4_유동인구_수,남성연령대_20_월요일시간대_5_유동인구_수,남성연령대_20_월요일시간대_6_유동인구_수,남성연령대_20_화요일시간대_1_유동인구_수,남성연령대_20_화요일시간대_2_유동인구_수,남성연령대_20_화요일시간대_3_유동인구_수,남성연령대_20_화요일시간대_4_유동인구_수,남성연령대_20_화요일시간대_5_유동인구_수,남성연령대_20_화요일시간대_6_유동인구_수,남성연령대_20_수요일시간대_1_유동인구_수,남성연령대_20_수요일시간대_2_유동인구_수,남성연령대_20_수요일시간대_3_유동인구_수,남성연령대_20_수요일시간대_4_유동인구_수,남성연령대_20_수요일시간대_5_유동인구_수,남성연령대_20_수요일시간대_6_유동인구_수,남성연령대_20_목요일시간대_1_유동인구_수,남성연령대_20_목요일시간대_2_유동인구_수,남성연령대_20_목요일시간대_3_유동인구_수,남성연령대_20_목요일시간대_4_유동인구_수,남성연령대_20_목요일시간대_5_유동인구_수,남성연령대_20_목요일시간대_6_유동인구_수,남성연령대_20_금요일시간대_1_유동인구_수,남성연령대_20_금요일시간대_2_유동인구_수,남성연령대_20_금요일시간대_3_유동인구_수,남성연령대_20_금요일시간대_4_유동인구_수,남성연령대_20_금요일시간대_5_유동인구_수,남성연령대_20_금요일시간대_6_유동인구_수,남성연령대_20_토요일시간대_1_유동인구_수,남성연령대_20_토요일시간대_2_유동인구_수,남성연령대_20_토요일시간대_3_유동인구_수,남성연령대_20_토요일시간대_4_유동인구_수,남성연령대_20_토요일시간대_5_유동인구_수,남성연령대_20_토요일시간대_6_유동인구_수,남성연령대_20_일요일시간대_1_유동인구_수,남성연령대_20_일요일시간대_2_유동인구_수,남성연령대_20_일요일시간대_3_유동인구_수,남성연령대_20_일요일시간대_4_유동인구_수,남성연령대_20_일요일시간대_5_유동인구_수,남성연령대_20_일요일시간대_6_유동인구_수,남성연령대_30_월요일시간대_1_유동인구_수,남성연령대_30_월요일시간대_2_유동인구_수,남성연령대_30_월요일시간대_3_유동인구_수,남성연령대_30_월요일시간대_4_유동인구_수,남성연령대_30_월요일시간대_5_유동인구_수,남성연령대_30_월요일시간대_6_유동인구_수,남성연령대_30_화요일시간대_1_유동인구_수,남성연령대_30_화요일시간대_2_유동인구_수,남성연령대_30_화요일시간대_3_유동인구_수,남성연령대_30_화요일시간대_4_유동인구_수,남성연령대_30_화요일시간대_5_유동인구_수,남성연령대_30_화요일시간대_6_유동인구_수,남성연령대_30_수요일시간대_1_유동인구_수,남성연령대_30_수요일시간대_2_유동인구_수,남성연령대_30_수요일시간대_3_유동인구_수,남성연령대_30_수요일시간대_4_유동인구_수,남성연령대_30_수요일시간대_5_유동인구_수,남성연령대_30_수요일시간대_6_유동인구_수,남성연령대_30_목요일시간대_1_유동인구_수,남성연령대_30_목요일시간대_2_유동인구_수,남성연령대_30_목요일시간대_3_유동인구_수,남성연령대_30_목요일시간대_4_유동인구_수,남성연령대_30_목요일시간대_5_유동인구_수,남성연령대_30_목요일시간대_6_유동인구_수,남성연령대_30_금요일시간대_1_유동인구_수,남성연령대_30_금요일시간대_2_유동인구_수,남성연령대_30_금요일시간대_3_유동인구_수,남성연령대_30_금요일시간대_4_유동인구_수,남성연령대_30_금요일시간대_5_유동인구_수,남성연령대_30_금요일시간대_6_유동인구_수,남성연령대_30_토요일시간대_1_유동인구_수,남성연령대_30_토요일시간대_2_유동인구_수,남성연령대_30_토요일시간대_3_유동인구_수,남성연령대_30_토요일시간대_4_유동인구_수,남성연령대_30_토요일시간대_5_유동인구_수,남성연령대_30_토요일시간대_6_유동인구_수,남성연령대_30_일요일시간대_1_유동인구_수,남성연령대_30_일요일시간대_2_유동인구_수,남성연령대_30_일요일시간대_3_유동인구_수,남성연령대_30_일요일시간대_4_유동인구_수,남성연령대_30_일요일시간대_5_유동인구_수,남성연령대_30_일요일시간대_6_유동인구_수,남성연령대_40_월요일시간대_1_유동인구_수,남성연령대_40_월요일시간대_2_유동인구_수,남성연령대_40_월요일시간대_3_유동인구_수,남성연령대_40_월요일시간대_4_유동인구_수,남성연령대_40_월요일시간대_5_유동인구_수,남성연령대_40_월요일시간대_6_유동인구_수,남성연령대_40_화요일시간대_1_유동인구_수,남성연령대_40_화요일시간대_2_유동인구_수,남성연령대_40_화요일시간대_3_유동인구_수,남성연령대_40_화요일시간대_4_유동인구_수,남성연령대_40_화요일시간대_5_유동인구_수,남성연령대_40_화요일시간대_6_유동인구_수,남성연령대_40_수요일시간대_1_유동인구_수,남성연령대_40_수요일시간대_2_유동인구_수,남성연령대_40_수요일시간대_3_유동인구_수,남성연령대_40_수요일시간대_4_유동인구_수,남성연령대_40_수요일시간대_5_유동인구_수,남성연령대_40_수요일시간대_6_유동인구_수,남성연령대_40_목요일시간대_1_유동인구_수,남성연령대_40_목요일시간대_2_유동인구_수,남성연령대_40_목요일시간대_3_유동인구_수,남성연령대_40_목요일시간대_4_유동인구_수,남성연령대

In [26]:
market_c_store.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수
0,201707,12763,곰달래로13길_B,CS300008,패션잡화,2,0,0.0,0,0.0,0


In [27]:
market_c_apartment.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,아파트_면적_132_제곱미터_세대_수,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,201707,13651,도봉로152가길,3,21.0,1.0,1.0,NaN,NaN,20.0,2.0,1.0,NaN,NaN,NaN,NaN,49.316498,7.406465e+07


In [28]:
market_c_indiIndex.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,과밀_지수_구간_코드,활성도_지표_구간_코드,성장성_지표_구간_코드,안정성_지표_구간_코드,과밀_지수_구간_코드_명,활성도_지표_구간_코드_명,성장성_지표_구간_코드_명,안정성_지표_구간_코드_명,과밀_지수_값,활성도_지표_값,성장성_지표_값,안정성_지표_값
0,201707,12727,구천면로_D,CS100003,일식집,2.0,3.0,1,4,경계,보통,높음,낮음,53.3,44.29,75.11,34.27


In [29]:
market_c_incomConsum.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,201707,12551,강서로_D,401,5,62338192,16799129,7171667,3981303,8086521,9160403,2002790,1793030,10880540,2462809


In [30]:
market_c_residPopulation.head(1)

,기준년월 코드,상권 코드,상권 코드 명,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,남성연령대 20 상주인구 수,남성연령대 30 상주인구 수,남성연령대 40 상주인구 수,남성연령대 50 상주인구 수,남성연령대 60 이상 상주인구 수,여성연령대 10 상주인구 수,여성연령대 20 상주인구 수,여성연령대 30 상주인구 수,여성연령대 40 상주인구 수,여성연령대 50 상주인구 수,여성연령대 60 이상 상주인구 수,총 가구 수,아파트 가구 수,비 아파트 가구 수
0,201707,13651,도봉로152가길,681,357,324,72,77,76,86,161,209,38,48,44,46,87,94,34,29,32,40,74,115,364.0,0,364.0


In [31]:
market_c_workPopulation.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,201707,13463,시흥대로_H,196,119,77,0,24,35,55,56,26,0,7,20,33,42,17,0,17,15,22,14,9


In [32]:
market_nc_store.head(1)

,기준 년월 코드,상권 코드,서비스 업종 코드,서비스 업종 코드명,점포 수,유사 업종 점포 수,개업율,개업 점포 수,폐업률,폐업 점포 수,상권 코드명
0,201706,13651,CS300011,화장품,3,0,0.0,0,0.0,0,도봉로152가길


In [33]:
market_nc_apartment.head(1)

,기준년월 코드,상권 코드,상권 코드명,아파트 단지 수,아파트 면적 66제곱미터 미만 세대 수,아파트 면적 66제곱미터 세대 수,아파트 면적 99제곱미터 세대 수,아파트 면적 132제곱미터 세대 수,아파트 면적 165제곱미터 세대 수,아파트 가격 1억 미만 세대 수,아파트 가격 1억 세대 수,아파트 가격 2억 세대 수,아파트 가격 3억 세대 수,아파트 가격 4억 세대 수,아파트 가격 5억 세대 수,아파트 가격 6억 이상 세대 수,아파트 평균 면적,아파트 평균 시가
0,201707,13651,도봉로152가길,316,4872.0,2680.0,413.0,38.0,2.0,1888.0,3360.0,2327.0,430.0,NaN,NaN,NaN,54.683572,1.052281e+08


In [34]:
market_nc_incomConsum.head(1)

,기준년월코드,상권코드,상권코드명,월평균소득 금액,소득구간코드,지출 총금액,식료품 지출 총금액,의류 신발 지출 총금액,생활용품 지출 총금액,의료비 지출 총금액,교통 지출 총금액,여가 지출 총금액,문화 지출 총금액,교육 지출 총금액,유흥 지출 총금액
0,201707,12300,면목로_H,466,6,15639040250,4455311095,1858160057,1147044872,2176405519,2442194645,535827978,500810554,1865538425,657747105


In [35]:
market_nc_residPopulation.head(1)

,기준년월 코드,상권 코드,총 상주인구 수,남성 상주인구 수,여성 상주인구 수,연령대 10 상주인구 수,연령대 20 상주인구 수,연령대 30 상주인구 수,연령대 40 상주인구 수,연령대 50 상주인구 수,연령대 60 이상 상주인구 수,남성연령대 10 상주인구 수,남성연령대 20 상주인구 수,남성연령대 30 상주인구 수,남성연령대 40 상주인구 수,남성연령대 50 상주인구 수,남성연령대 60 이상 상주인구 수,여성연령대 10 상주인구 수,여성연령대 20 상주인구 수,여성연령대 30 상주인구 수,여성연령대 40 상주인구 수,여성연령대 50 상주인구 수,여성연령대 60 이상 상주인구 수,총 가구 수,아파트 가구 수,비 아파트 가구 수,상권 코드 명
0,201707,13651,30573,15360,15213,4518,3728,4394,4828,5207,7898,2353,1968,2283,2549,2557,3650,2165,1760,2111,2279,2650,4248,12760.0,5124,7636.0,도봉로152가길


In [36]:
market_nc_expSale.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,평균_영업_개월_수,1년이하_생존_율,1~2년_생존_율,2~3년_생존_율,3~5년_생존_율,5년이상_생존_율,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,201706,13651,도봉로152가길,CS300011,화장품,405.0,0.0,0.0,0.0,0.0,0.0,14761198,231,80.5,19.5,8.2,13.0,17.3,29.0,13.0,19.5,0.0,4.8,5.6,26.4,23.8,32.9,6.5,28.4,71.6,0.0,13.3,12.0,22.2,33.8,18.7,12738112,2023086,1811867,1268782,1369023,6851324,1437116,2023086,0,1294117,340625,5132335,3226271,2562049,2205801,4096029,9438175,0,2123819,1516252,3735119,5365506,793508,186,45,19,30,40,67,30,45,0,11,13,61,55,76,15,64,161,0,30,27,50,76,42,7


In [37]:
market_nc_workPopulation.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,201707,12071,동일로_L,429,258,171,0,37,49,96,161,86,0,11,30,41,105,71,0,26,19,55,56,15


In [38]:
market_nc_attractFacility.head(1)

,기준_년월_코드,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,201707,12679,목동중앙북로16길,94,3.0,3.0,NaN,2.0,12,4.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,1.0,20.0


In [39]:
market_nc_expFloatingPopulation.head(1)

,기준_년월_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,시간대_1_유동인구_수,시간대_2_유동인구_수,시간대_3_유동인구_수,시간대_4_유동인구_수,시간대_5_유동인구_수,시간대_6_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수,남성연령대_10_월요일시간대_1_유동인구_수,남성연령대_10_월요일시간대_2_유동인구_수,남성연령대_10_월요일시간대_3_유동인구_수,남성연령대_10_월요일시간대_4_유동인구_수,남성연령대_10_월요일시간대_5_유동인구_수,남성연령대_10_월요일시간대_6_유동인구_수,남성연령대_10_화요일시간대_1_유동인구_수,남성연령대_10_화요일시간대_2_유동인구_수,남성연령대_10_화요일시간대_3_유동인구_수,남성연령대_10_화요일시간대_4_유동인구_수,남성연령대_10_화요일시간대_5_유동인구_수,남성연령대_10_화요일시간대_6_유동인구_수,남성연령대_10_수요일시간대_1_유동인구_수,남성연령대_10_수요일시간대_2_유동인구_수,남성연령대_10_수요일시간대_3_유동인구_수,남성연령대_10_수요일시간대_4_유동인구_수,남성연령대_10_수요일시간대_5_유동인구_수,남성연령대_10_수요일시간대_6_유동인구_수,남성연령대_10_목요일시간대_1_유동인구_수,남성연령대_10_목요일시간대_2_유동인구_수,남성연령대_10_목요일시간대_3_유동인구_수,남성연령대_10_목요일시간대_4_유동인구_수,남성연령대_10_목요일시간대_5_유동인구_수,남성연령대_10_목요일시간대_6_유동인구_수,남성연령대_10_금요일시간대_1_유동인구_수,남성연령대_10_금요일시간대_2_유동인구_수,남성연령대_10_금요일시간대_3_유동인구_수,남성연령대_10_금요일시간대_4_유동인구_수,남성연령대_10_금요일시간대_5_유동인구_수,남성연령대_10_금요일시간대_6_유동인구_수,남성연령대_10_토요일시간대_1_유동인구_수,남성연령대_10_토요일시간대_2_유동인구_수,남성연령대_10_토요일시간대_3_유동인구_수,남성연령대_10_토요일시간대_4_유동인구_수,남성연령대_10_토요일시간대_5_유동인구_수,남성연령대_10_토요일시간대_6_유동인구_수,남성연령대_10_일요일시간대_1_유동인구_수,남성연령대_10_일요일시간대_2_유동인구_수,남성연령대_10_일요일시간대_3_유동인구_수,남성연령대_10_일요일시간대_4_유동인구_수,남성연령대_10_일요일시간대_5_유동인구_수,남성연령대_10_일요일시간대_6_유동인구_수,남성연령대_20_월요일시간대_1_유동인구_수,남성연령대_20_월요일시간대_2_유동인구_수,남성연령대_20_월요일시간대_3_유동인구_수,남성연령대_20_월요일시간대_4_유동인구_수,남성연령대_20_월요일시간대_5_유동인구_수,남성연령대_20_월요일시간대_6_유동인구_수,남성연령대_20_화요일시간대_1_유동인구_수,남성연령대_20_화요일시간대_2_유동인구_수,남성연령대_20_화요일시간대_3_유동인구_수,남성연령대_20_화요일시간대_4_유동인구_수,남성연령대_20_화요일시간대_5_유동인구_수,남성연령대_20_화요일시간대_6_유동인구_수,남성연령대_20_수요일시간대_1_유동인구_수,남성연령대_20_수요일시간대_2_유동인구_수,남성연령대_20_수요일시간대_3_유동인구_수,남성연령대_20_수요일시간대_4_유동인구_수,남성연령대_20_수요일시간대_5_유동인구_수,남성연령대_20_수요일시간대_6_유동인구_수,남성연령대_20_목요일시간대_1_유동인구_수,남성연령대_20_목요일시간대_2_유동인구_수,남성연령대_20_목요일시간대_3_유동인구_수,남성연령대_20_목요일시간대_4_유동인구_수,남성연령대_20_목요일시간대_5_유동인구_수,남성연령대_20_목요일시간대_6_유동인구_수,남성연령대_20_금요일시간대_1_유동인구_수,남성연령대_20_금요일시간대_2_유동인구_수,남성연령대_20_금요일시간대_3_유동인구_수,남성연령대_20_금요일시간대_4_유동인구_수,남성연령대_20_금요일시간대_5_유동인구_수,남성연령대_20_금요일시간대_6_유동인구_수,남성연령대_20_토요일시간대_1_유동인구_수,남성연령대_20_토요일시간대_2_유동인구_수,남성연령대_20_토요일시간대_3_유동인구_수,남성연령대_20_토요일시간대_4_유동인구_수,남성연령대_20_토요일시간대_5_유동인구_수,남성연령대_20_토요일시간대_6_유동인구_수,남성연령대_20_일요일시간대_1_유동인구_수,남성연령대_20_일요일시간대_2_유동인구_수,남성연령대_20_일요일시간대_3_유동인구_수,남성연령대_20_일요일시간대_4_유동인구_수,남성연령대_20_일요일시간대_5_유동인구_수,남성연령대_20_일요일시간대_6_유동인구_수,남성연령대_30_월요일시간대_1_유동인구_수,남성연령대_30_월요일시간대_2_유동인구_수,남성연령대_30_월요일시간대_3_유동인구_수,남성연령대_30_월요일시간대_4_유동인구_수,남성연령대_30_월요일시간대_5_유동인구_수,남성연령대_30_월요일시간대_6_유동인구_수,남성연령대_30_화요일시간대_1_유동인구_수,남성연령대_30_화요일시간대_2_유동인구_수,남성연령대_30_화요일시간대_3_유동인구_수,남성연령대_30_화요일시간대_4_유동인구_수,남성연령대_30_화요일시간대_5_유동인구_수,남성연령대_30_화요일시간대_6_유동인구_수,남성연령대_30_수요일시간대_1_유동인구_수,남성연령대_30_수요일시간대_2_유동인구_수,남성연령대_30_수요일시간대_3_유동인구_수,남성연령대_30_수요일시간대_4_유동인구_수,남성연령대_30_수요일시간대_5_유동인구_수,남성연령대_30_수요일시간대_6_유동인구_수,남성연령대_30_목요일시간대_1_유동인구_수,남성연령대_30_목요일시간대_2_유동인구_수,남성연령대_30_목요일시간대_3_유동인구_수,남성연령대_30_목요일시간대_4_유동인구_수,남성연령대_30_목요일시간대_5_유동인구_수,남성연령대_30_목요일시간대_6_유동인구_수,남성연령대_30_금요일시간대_1_유동인구_수,남성연령대_30_금요일시간대_2_유동인구_수,남성연령대_30_금요일시간대_3_유동인구_수,남성연령대_30_금요일시간대_4_유동인구_수,남성연령대_30_금요일시간대_5_유동인구_수,남성연령대_30_금요일시간대_6_유동인구_수,남성연령대_30_토요일시간대_1_유동인구_수,남성연령대_30_토요일시간대_2_유동인구_수,남성연령대_30_토요일시간대_3_유동인구_수,남성연령대_30_토요일시간대_4_유동인구_수,남성연령대_30_토요일시간대_5_유동인구_수,남성연령대_30_토요일시간대_6_유동인구_수,남성연령대_30_일요일시간대_1_유동인구_수,남성연령대_30_일요일시간대_2_유동인구_수,남성연령대_30_일요일시간대_3_유동인구_수,남성연령대_30_일요일시간대_4_유동인구_수,남성연령대_30_일요일시간대_5_유동인구_수,남성연령대_30_일요일시간대_6_유동인구_수,남성연령대_40_월요일시간대_1_유동인구_수,남성연령대_40_월요일시간대_2_유동인구_수,남성연령대_40_월요일시간대_3_유동인구_수,남성연령대_40_월요일시간대_4_유동인구_수,남성연령대_40_월요일시간대_5_유동인구_수,남성연령대_40_월요일시간대_6_유동인구_수,남성연령대_40_화요일시간대_1_유동인구_수,남성연령대_40_화요일시간대_2_유동인구_수,남성연령대_40_화요일시간대_3_유동인구_수,남성연령대_40_화요일시간대_4_유동인구_수,남성연령대_40_화요일시간대_5_유동인구_수,남성연령대_40_화요일시간대_6_유동인구_수,남성연령대_40_수요일시간대_1_유동인구_수,남성연령대_40_수요일시간대_2_유동인구_수,남성연령대_40_수요일시간대_3_유동인구_수,남성연령대_40_수요일시간대_4_유동인구_수,남성연령대_40_수요일시간대_5_유동인구_수,남성연령대_40_수요일시간대_6_유동인구_수,남성연령대_40_목요일시간대_1_유동인구_수,남성연령대_40_목요일시간대_2_유동인구_수,남성연령대_40_목요일시간대_3_유동인구_수,남성연령대_40_목요일시간대_4_유동인구_수,남성연령대_40_목요일시